<a href="https://colab.research.google.com/github/Facco-Bruno/RAG/blob/Master/RAG_Text_to_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## RAG (Retrieval Augmented Generation)

### APPLICATION - TEXT - TO - SQL GENERATOR

Conforme a figura 1, a ideia é desenvolver uma aplicação do tipo Text - to - SQL, podendo ser feito de diferentes formas e utilizando diferentes ferramentas/modelos. Devido a performance e também facilidade de utilização, foi utilizado a API da OPEN AI para geração de dois exemplos, um construindo uma ferramenta para aprendizado da linguagem SQL, e um aplicando gerador de querys e respostas a partir de bases de dados adicionadas no sistema.

#### EXEMPLO USANDO LLAMA INDEX COM GPT

##### PARTE 1: QUERY ENGINE

In [ ]:
!pip install -q llama-index-llms-openai
!pip install -q llama-index
!pip install -q cohere
!pip install -q openai

In [ ]:
import os
import openai

In [ ]:
os.environ["OPENAI_API_KEY"] = "XXXXXXXXXXXXXXXXXXXXX"
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
from IPython.display import Markdown, display

In [ ]:
from sqlalchemy import (create_engine, MetaData, Table, Column, String, Integer,select)

In [ ]:
from llama_index.core import SQLDatabase
from llama_index.llms.openai import OpenAI

In [ ]:
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

In [ ]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)
metadata_obj.create_all(engine)

In [ ]:
llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")

In [ ]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])

In [ ]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])
from sqlalchemy import insert

rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {"city_name": "Chicago", "population": 2679000,"country": "United States"},
    {"city_name": "Seoul", "population": 9776000, "country": "South Korea"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [ ]:
# view current table
stmt = select(
    city_stats_table.c.city_name,
    city_stats_table.c.population,
    city_stats_table.c.country,
).select_from(city_stats_table)

with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()
    print(results)

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Chicago', 2679000, 'United States'), ('Seoul', 9776000, 'South Korea')]


In [ ]:
from sqlalchemy import text

with engine.connect() as con:
    rows = con.execute(text("SELECT city_name from city_stats"))
    for row in rows:
        print(row)

('Chicago',)
('Seoul',)
('Tokyo',)
('Toronto',)


In [ ]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database, tables=["city_stats"], llm=llm
)

query_str = "Which city has the highest population?"
response = query_engine.query(query_str)

In [ ]:
display(Markdown(f"<b>{response}</b>"))

<b>The city with the highest population is Tokyo.</b>

##### PARTE 2 - QUERY TIME RETRIEVAL

In [ ]:
from llama_index.core.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine,
)
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import VectorStoreIndex

# set Logging to DEBUG for more detailed outputs
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="city_stats"))
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=1)
)

In [ ]:
response = query_engine.query("Which city has the highest population?")
display(Markdown(f"<b>{response}</b>"))

<b>The city with the highest population is Tokyo.</b>

In [ ]:
response.metadata["result"]

[('Tokyo',)]

In [ ]:
city_stats_text = (
    "This table gives information regarding the population and country of a"
    " given city.\nThe user will query with codewords, where 'foo' corresponds"
    " to population and 'bar'corresponds to city."
)

table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="city_stats", context_str=city_stats_text))
]

##### PARTE 3: TEXT-TO-SQL RETRIEVER

In [ ]:
from llama_index.core.retrievers import NLSQLRetriever

# default retrieval (return_raw=True)
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["city_stats"], return_raw=True
)

In [ ]:
results = nl_sql_retriever.retrieve(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [ ]:
from llama_index.core.response.notebook_utils import display_source_node

for n in results:
    display_source_node(n)

**Node ID:** 84c6c6b3-2215-44a9-88ab-c69a5c111163<br>**Similarity:** None<br>**Text:** [('Tokyo', 13960000), ('Seoul', 9776000), ('Toronto', 2930000), ('Chicago', 2679000)]<br>

In [ ]:
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["city_stats"], return_raw=False
)

In [ ]:
results = nl_sql_retriever.retrieve(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [ ]:
for n in results:
    display_source_node(n, show_source_metadata=True)

**Node ID:** a91b7248-7960-4002-932e-2acb31f403f3<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Tokyo', 'population': 13960000}<br>

**Node ID:** 4b18d4fe-610d-4d5b-9c3f-b274f21e05f2<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Seoul', 'population': 9776000}<br>

**Node ID:** 84fa65f1-0d70-4e2d-90ca-4fb93e27e21b<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Toronto', 'population': 2930000}<br>

**Node ID:** e7f80741-7358-410a-945c-f595048282ef<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Chicago', 'population': 2679000}<br>

In [ ]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(nl_sql_retriever)

In [ ]:
response = query_engine.query(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [ ]:
print(str(response))

The top 5 cities with the highest population are:

1. Tokyo - 13,960,000
2. Seoul - 9,776,000
3. Toronto - 2,930,000
4. Chicago - 2,679,000


#### APLICAÇÃO COM LLAMA INDEX USANDO MAIS DE UMA TABELA PARA GERAR QUERYS

In [ ]:
!pip install -q llama-index-llms-openai
!pip install -q llama-index
!pip install -q cohere
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.4/262.4 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 49.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.8/30.8 MB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from llama_index.core import SQLDatabase
from llama_index.llms.openai import OpenAI
from sqlalchemy import insert
import openai
import os
from sqlalchemy import (create_engine,MetaData,Table,Column,String,Integer,select)
from IPython.display import Markdown, display

In [ ]:
os.environ["OPENAI_API_KEY"] = "xxxxxxxxxxxxxxx"
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")

In [ ]:
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

##### PARTE 1 - ENGINE QUERY

###### TABELA 1

In [ ]:
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
    Column("continent", String(16), nullable=False),
)
metadata_obj.create_all(engine)

In [ ]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])

In [ ]:
rows = [
    {"city_name": "Toronto", "population": 293, "country": "Canada", "continent" : 'America'},
    {"city_name": "Quebec", "population": 247, "country": "Canada", "continent" : 'America'},
    {"city_name": "Valencia", "population": 167, "country": "Spain", "continent" : 'Europe'},
    {"city_name": "Paris", "population": 189, "country": "France", "continent" : 'Europe'},
    {"city_name": "Tokyo", "population": 139, "country": "Japan", "continent" : 'Asia'},
    {"city_name": "Chicago", "population": 267,"country": "United States", "continent" : 'America'},
    {"city_name": "Washington", "population": 250,"country": "United States", "continent" : 'America'},
    {"city_name": "Seoul", "population": 97, "country": "South Korea", "continent" : 'Asia'},
]

for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [ ]:
stmt = select(
    city_stats_table.c.city_name,
    city_stats_table.c.population,
    city_stats_table.c.country,
).select_from(city_stats_table)

with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()
    print(results)

[('Toronto', 293, 'Canada'), ('Quebec', 247, 'Canada'), ('Valencia', 167, 'Spain'), ('Paris', 189, 'France'), ('Tokyo', 139, 'Japan'), ('Chicago', 267, 'United States'), ('Washington', 250, 'United States'), ('Seoul', 97, 'South Korea')]


###### TABELA 2

In [ ]:
table_name = "city_economy"
city_economy_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("pib", Integer),
    Column("rank", Integer),
    Column("mean_industry", String(16), nullable=False),
)
metadata_obj.create_all(engine)

In [ ]:
sql_database = SQLDatabase(engine, include_tables=["city_economy"])

In [ ]:
rows = [
    {"city_name": "Toronto", "pib": 120, "rank": 3, "mean_industry": 'Technology'},
    {"city_name": "Quebec", "pib": 90, "rank" : 7, "mean_industry": 'Technology'},
    {"city_name": "Valencia", "pib": 79, "rank" : 12, "mean_industry": 'Tourism'},
    {"city_name": "Paris", "pib": 85, "rank": 10, "mean_industry": 'Commerce'},
    {"city_name": "Tokyo", "pib": 190, "rank": 1, "mean_industry": 'Technology'},
    {"city_name": "Chicago", "pib": 110, "rank": 4, "mean_industry": 'Finance'},
    {"city_name": "Washington", "pib": 100, "rank": 5, "mean_industry": 'Commerce'},
    {"city_name": "Seoul", "pib": 145, "rank": 2, "mean_industry": 'Technology'},
]
for row in rows:
    stmt = insert(city_economy_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [ ]:
stmt = select(
    city_economy_table.c.city_name,
    city_economy_table.c.pib,
    city_economy_table.c.rank,
    city_economy_table.c.mean_industry,
).select_from(city_economy_table)

with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()
    print(results)

[('Toronto', 120, 3, 'Technology'), ('Quebec', 90, 7, 'Technology'), ('Valencia', 79, 12, 'Tourism'), ('Paris', 85, 10, 'Commerce'), ('Tokyo', 190, 1, 'Technology'), ('Chicago', 110, 4, 'Finance'), ('Washington', 100, 5, 'Commerce'), ('Seoul', 145, 2, 'Technology')]


###### TESTANDO

In [ ]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database, tables=["city_stats", "city_economy"], llm=llm
)

In [ ]:
query_str = "Which is the mean industry from the city with the lowest population?"
response = query_engine.query(query_str)

In [ ]:
print(response)

The city with the lowest population is Seoul, and the mean industry in that city is technology.


In [ ]:
print(response.metadata['sql_query'])

SELECT city_name, mean_industry 
FROM city_economy 
WHERE city_name = (SELECT city_name 
                   FROM city_stats 
                   ORDER BY population ASC 
                   LIMIT 1)


In [ ]:
query_str = "Which is the total pib from the city with the highest rank in America?"
response = query_engine.query(query_str)

In [ ]:
print(response)

The query did not return any results for the city with the highest rank in America. It is possible that there was an error in the data or the city with the highest rank in America does not have its PIB information available in the database.


In [ ]:
print(response.metadata['sql_query'])

SELECT ce.city_name, ce.pib
FROM city_economy ce
WHERE ce.rank = (SELECT MAX(rank) FROM city_economy WHERE mean_industry = 'America')


In [ ]:
query_str = "What is the mean industry from the highest pib in asia"
response = query_engine.query(query_str)

In [ ]:
print(response)

The mean industry from the city with the highest GDP in Asia is Technology.


In [ ]:
print(response.metadata['sql_query'])

SELECT ce.mean_industry
FROM city_economy ce
JOIN city_stats cs ON ce.city_name = cs.city_name
WHERE cs.continent = 'Asia'
ORDER BY ce.pib DESC
LIMIT 1;


##### PARTE 2 - QUERY TIME RETRIEVAL

In [ ]:
from llama_index.core.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine,
)
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import VectorStoreIndex

# set Logging to DEBUG for more detailed outputs
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="city_stats")), (SQLTableSchema(table_name="city_economy"))
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=1)
)

In [ ]:
response = query_engine.query("Which city has the highest population?")
print(response)

Toronto has the highest population among the cities listed in the database, with a population of 293.


In [ ]:
response = query_engine.query("Which city has the highest pi?")
print(response)

Tokyo has the highest PIB (GDP) among all cities, with a PIB of 190.


In [ ]:
city_stats_text = (
    "This table gives information regarding the population and country of a"
    " given city.\nThe user will query with codewords, where 'foo' corresponds"
    " to population and 'bar'corresponds to city."
)

city_economy_text = (
    "This table gives information regarding the economy and rank of a"
    " given city.\nThe user will query with codewords, where 'foo' corresponds"
    " to population and 'bar'corresponds to city."
)

table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="city_stats", context_str=city_stats_text)), (SQLTableSchema(table_name="city_economy", context_str=city_economy_text))
]

##### PARTE 3 - TEXT - TO - SQL RETRIEVER

In [ ]:
from llama_index.core.retrievers import NLSQLRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["city_stats", "city_economy"], return_raw=True
)

In [ ]:
query_engine = RetrieverQueryEngine.from_args(nl_sql_retriever)

In [ ]:
response = query_engine.query(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [ ]:
print(str(response))

Return the top 5 cities (along with their populations) with the highest population:
1. Toronto - 293
2. Chicago - 267
3. Washington - 250
4. Quebec - 247
5. Paris - 189


In [ ]:
response = query_engine.query(
    "Return the top 5 cities (along with their pib) with the highest pib."
)

In [ ]:
print(str(response))

Return the top 5 cities with the highest PIB are Tokyo (190), Seoul (145), Toronto (120), Chicago (110), and Washington (100).


In [ ]:
response = query_engine.query(
    "Return the top 5 cities (along with their pib and population) with the highest pib."
)

In [ ]:
print(str(response))

The top 5 cities with the highest PIB are:
1. Tokyo - PIB: 190, Population: 139
2. Seoul - PIB: 145, Population: 97
3. Toronto - PIB: 120, Population: 293
4. Chicago - PIB: 110, Population: 267
5. Washington - PIB: 100, Population: 250


In [ ]:
response = query_engine.query(
    "Return the top 3 countries (along with their cities and pib) with the highest pib."
)

In [ ]:
print(str(response))

Return Tokyo (Technology) with a PIB of 190, Seoul (Technology) with a PIB of 145, and Toronto (Technology) with a PIB of 120 as the top 3 countries with the highest PIB.


##### PARTE 4 - USANDO MECANISMOS DE VALIDAÇÃO DAS QUERYS

In [ ]:
!pip install -q evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00


PERGUNTA

In [ ]:
query_str = "Which is the population sum from the countries in America?"
response = query_engine.query(query_str)

In [ ]:
print(response)

The population sum from the countries in America is 1,057.


In [ ]:
query_resposta = response.metadata['sql_query']

In [ ]:
query_resposta

"SELECT SUM(population) \nFROM city_stats \nWHERE continent = 'America'"

In [ ]:
print(query_resposta)

SELECT SUM(population) 
FROM city_stats 
WHERE continent = 'America'


RESPOSTA ESPERADA

In [ ]:
query_referencia = "SELECT SUM(population) FROM city_stats WHERE continent = 'America'"

###### BLEU

In [ ]:
import evaluate

# Define the candidate predictions and reference sentences
predictions = [query_resposta]
references = [query_referencia]

# Load the BLEU evaluation metric
bleu = evaluate.load("bleu")

# Compute the BLEU score
results = bleu.compute(predictions=predictions, references=references)

# Print the results
print(results)

{'bleu': 1.0, 'precisions': [1.0, 1.0, 1.0, 1.0], 'brevity_penalty': 1.0, 'length_ratio': 1.0, 'translation_length': 13, 'reference_length': 13}


###### ROUGE

In [ ]:
pip install -q rouge_score

In [ ]:
import evaluate

# Load the ROUGE evaluation metric
rouge = evaluate.load('rouge')

# Define the candidate predictions and reference sentences
predictions = [query_resposta]
references = [query_referencia]

# Compute the ROUGE score
results = rouge.compute(predictions=predictions, references=references)

# Print the results
print(results)

{'rouge1': 1.0, 'rouge2': 1.0, 'rougeL': 1.0, 'rougeLsum': 1.0}


#### USANDO BASE DE DADOS DO SPIDER E CONECTANDO COM SQLITE

In [ ]:
!pip install -q llama-index-llms-openai
!pip install -q llama-index
!pip install -q cohere
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 22.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 44.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 41.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.5/525.5 kB 38.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from llama_index.core import SQLDatabase
from llama_index.llms.openai import OpenAI
from sqlalchemy import insert
import openai
import os
from sqlalchemy import (inspect, create_engine,MetaData,Table,Column,String,Integer,select, text)
from IPython.display import Markdown, display
import pandas as pd

In [ ]:
os.environ["OPENAI_API_KEY"] = "XXXXXXXXXXXXXXX"
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
llm = OpenAI(temperature=0.1, model="gpt-3.5-turbo")

##### Pegar caminho das bases do sqlite

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
base_wta = 'Caminho/wta_1.sqlite'

In [ ]:
base_baseball = 'Caminho/baseball_1.sqlite'

##### Acessar base do SQLite

In [ ]:
engine = create_engine("sqlite:///"+base_wta)

In [ ]:
table_names = inspect(engine).get_table_names()
print(table_names)

['matches', 'players', 'rankings']


In [ ]:
sql_database = SQLDatabase(engine, include_tables=["matches", "players", "rankings"])

##### Adicionar bases dentro da ferramenta do SQL

In [ ]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(sql_database=sql_database, tables=["matches", "players", "rankings"], llm=llm)

##### PARTE 1 - ENGINE QUERY

In [ ]:
query_str = "List the 10 players with minimum ranking points?"
response = query_engine.query(query_str)

print(response)
print('\n')
print(response.metadata['sql_query'])

The 10 players with the minimum ranking points are Svenja Weidemann and Virginie Ayassamy, each with 0 ranking points.


SELECT p.first_name, p.last_name, r.ranking_points
FROM players p
JOIN rankings r ON p.player_id = r.player_id
ORDER BY r.ranking_points
LIMIT 10;


In [ ]:
query_str = "List the 5 tournament names and countries with most competitions?"
response = query_engine.query(query_str)

print(response)
print('\n')
print(response.metadata['sql_query'])

The top 5 tournament names and countries with the most competitions are Indian Wells in Russia with 10 competitions, US Open in the USA with 7 competitions, Cincinnati in Spain with 6 competitions, Wimbledon in the USA with 6 competitions, and Australian Open in the USA with 5 competitions.


SELECT tourney_name, winner_ioc, COUNT(*) as num_competitions
FROM matches
GROUP BY tourney_name, winner_ioc
ORDER BY num_competitions DESC
LIMIT 5;


In [ ]:
query_str = "What is the median time from the players that plays with the Right Hand?"
response = query_engine.query(query_str)

print(response)
print('\n')
print(response.metadata['sql_query'])

The median time from players who play with their right hand is 95 minutes.


SELECT minutes
FROM matches
WHERE winner_hand = 'R' OR loser_hand = 'R'
ORDER BY minutes
LIMIT 1 OFFSET (SELECT COUNT(*) FROM matches WHERE winner_hand = 'R' OR loser_hand = 'R') / 2


In [ ]:
query_str = "What is the 5 countries with most tours by player?"
response = query_engine.query(query_str)

print(response)
print('\n')
print(response.metadata['sql_query'])

The top 5 countries with the most tours by players are the USA with 43,331 tours, Russia with 38,617 tours, Italy with 28,186 tours, Japan with 27,031 tours, and France with 25,138 tours.


SELECT p.country_code, COUNT(r.tours) AS total_tours
FROM players p
JOIN rankings r ON p.player_id = r.player_id
GROUP BY p.country_code
ORDER BY total_tours DESC
LIMIT 5;


In [ ]:
query_str = "What is year with most matches?"
response = query_engine.query(query_str)

print(response)
print('\n')
print(response.metadata['sql_query'])

The year with the most matches is 2016, with a total of 158 matches.


SELECT year, COUNT(*) AS match_count
FROM matches
GROUP BY year
ORDER BY match_count DESC
LIMIT 1;


In [ ]:
query_str = "What is the top player from every country?"
response = query_engine.query(query_str)

print(response)
print('\n')
print(response.metadata['sql_query'])

The top players from various countries include Victoria Azarenka from Belarus, Karolina Pliskova from the Czech Republic, Caroline Wozniacki from Denmark, Garbine Muguruza from Spain, Angelique Kerber from Germany, Maria Sharapova from Russia, Ana Ivanovic and Jelena Jankovic from Serbia, and Serena Williams from the USA.


SELECT p.country_code, p.first_name, p.last_name, r.ranking
FROM players p
JOIN rankings r ON p.player_id = r.player_id
WHERE r.ranking = 1
ORDER BY p.country_code;


##### PARTE 2 - QUERY TIME RETRIEVER

In [ ]:
from llama_index.core.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine,
)
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import VectorStoreIndex

# set Logging to DEBUG for more detailed outputs
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="matches")), (SQLTableSchema(table_name="players")), (SQLTableSchema(table_name="rankings"))
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=1)
)

In [ ]:
query_str = "What is the top player in winner rank from every country, list all?"
response = query_engine.query(query_str)

print(response)
print('\n')
print(response.metadata['sql_query'])

Apologies, there seems to be an error in the SQL query provided. Let me correct it for you.


SELECT p.country, p.player_name, r.ranking
FROM players p
JOIN rankings r ON p.player_id = r.player_id
WHERE r.ranking = 1
ORDER BY p.country;


In [ ]:
matches_text = (
    "This table gives information regarding the matches, stats and players information of a"
    " given player that win and a player who lose.\nThe user will query with codewords, where 'foo' corresponds"
    " to player and 'bar'corresponds to matchs and players information."
)

players_text = (
    "This table gives information regarding the characteristics of a"
    " given player.\nThe user will query with codewords, where 'foo' corresponds"
    " to player id and 'bar'corresponds to player information."
)

rankings_text = (
    "This table gives information regarding the players rank of a"
    " given player.\nThe user will query with codewords, where 'foo' corresponds"
    " to player id and 'bar'corresponds to player rank information."
)

table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="matches", context_str=matches_text)),
    (SQLTableSchema(table_name="players", context_str=players_text)),
    (SQLTableSchema(table_name="rankings", context_str=rankings_text))
]

In [ ]:
query_str = "What is the the top player by ranking seed in each country code, list all countries and player first name?"
response = query_engine.query(query_str)

print(response)
print('\n')
print(response.metadata['sql_query'])

The top player by ranking seed in each country code are as follows:
- Belarus (BLR): Victoria
- Czech Republic (CZE): Karolina
- Denmark (DEN): Caroline
- Spain (ESP): Garbine
- Germany (GER): Angelique
- Russia (RUS): Maria
- Serbia (SRB): Ana and Jelena
- United States (USA): Serena


SELECT p.first_name, p.country_code, r.ranking
FROM rankings r
JOIN players p ON r.player_id = p.player_id
WHERE r.ranking = 1
ORDER BY p.country_code;


##### PARTE 3 - TEXT TO SQL RETRIEVER

In [ ]:
from llama_index.core.retrievers import NLSQLRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

nl_sql_retriever = NLSQLRetriever(sql_database, tables=["matches", "players", "rankings"], return_raw=True)

In [ ]:
query_engine = RetrieverQueryEngine.from_args(nl_sql_retriever)

In [ ]:
query_str = "What is the the top player by ranking seed in each country code, list all countries and player first name?"
response = query_engine.query(query_str)

print(str(response))

The top player by ranking seed in each country code is as follows:

- Country Code: US, Player First Name: John
- Country Code: UK, Player First Name: Emily
- Country Code: FR, Player First Name: Pierre


In [ ]:
print(str(response.source_nodes[0]))

Node ID: da5d4a3d-4f07-401b-af64-bc9b6293d63b
Text: Error: Statement 'SELECT p.country_code, p.first_name\nFROM
players p\nJOIN (\n    SELECT country_code, MAX(winner_seed) AS
max_seed\n    FROM matches\n    GROUP BY country_code\n) m ON
p.player_id = (\n    SELECT winner_id\n    FROM matches\n    WHERE
country_code = m.country_code AND winner_seed = m.max_seed\n)\nORDER
BY p.country_code;' is in...
Score: None



##### PARTE 4 - MÉTRICAS PARA VALIDAR AS QUERYS

CRIAR UMA BASE DE PERGUNTAS E QUERYS ESPERADAS PARA AQUELA PERGUNTA E USAR COMO BASE DE REFERÊNCIA PARA COMPARAR COM A RESPOSTA DO MODELO E ESCOLHER O SCORE QUE FAZ MAIS SENTIDO

In [ ]:
!pip install -q evaluate

###### BLEU

In [ ]:
import evaluate

# Define the candidate predictions and reference sentences
predictions = ["hello there general kenobi", "foo bar foobar"]
references = [["hello there general kenobi", "hello there !"],["foo bar foobar"]]

# Load the BLEU evaluation metric
bleu = evaluate.load("bleu")

# Compute the BLEU score
results = bleu.compute(predictions=predictions, references=references)

# Print the results
print(results)

###### ROUGE

In [ ]:
import evaluate

# Load the ROUGE evaluation metric
rouge = evaluate.load('rouge')

# Define the candidate predictions and reference sentences
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]

# Compute the ROUGE score
results = rouge.compute(predictions=predictions, references=references)

# Print the results
print(results)

#### USANDO O MODO DE CONVERSA DA OPEN AI PRA CONSTRUIR QUERYS (PASSANDO ESTRUTURAS DE TABELAS COMO CONTEXTO)

In [ ]:
!pip install -q openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.8/77.8 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.6 MB/s eta 0:00:00


In [ ]:
import os
from openai import OpenAI

In [ ]:
os.environ["OPENAI_API_KEY"] = "XXXXXXXXXXXXXXXXXXXXXXXXXXX"
openai.api_key = os.environ["OPENAI_API_KEY"]

In [ ]:
client = OpenAI()

In [ ]:
def continue_conversation(messages, temperature=0):
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=temperature,
    )
    return response.choices[0].message.content

In [ ]:
context = [ {'role':'system',
             'content':"""you are a bot to assist in create SQL commands, all your answers should start with \
              this is your SQL, and after that an SQL that can do what the user request. \
              Your Database is composed by a SQL database with some tables. \
              Try to Maintain the SQL order simple.
              Put the SQL command in white letters with a black background, and just after \
              a simple and concise text explaining how it works.
              If the user ask for something that can not be solved with an SQL Order \
              just answer something nice and simple, maximum 10 words, asking him for something that \
              can be solved with SQL.
              """}
            ]

context.append( {'role':'system',
                 'content':"""
first table:
{
  "tableName": "employees",
  "fields": [
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "name",
      "type": "string"
    }
  ]
}
"""
})

context.append( {'role':'system', 'content':"""
second table:
{
  "tableName": "salary",
  "fields": [
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "year",
      "type": "date"
    },
    {
      "name": "salary",
      "type": "float"
    }
  ]
}
"""
})

context.append( {'role':'system', 'content':"""
third table:
{
  "tablename": "studies",
  "fields": [
    {
      "name": "ID",
      "type": "int"
    },
    {
      "name": "ID_usr",
      "type": "int"
    },
    {
      "name": "educational level",
      "type": "int"
    },
    {
      "name": "Institution",
      "type": "string"
    },
    {
      "name": "Years",
      "type": "date"
    }
    {
      "name": "Speciality",
      "type": "string"
    }
  ]
}
"""
})

In [ ]:
prompt = 'give me the name of the 5 employees with higher salary'

In [ ]:
context.append({'role':'user', 'content':f"{prompt}."})
response = continue_conversation(context)

In [ ]:
print(response)

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN salary s ON e.ID_usr = s.ID_usr
ORDER BY s.salary DESC
LIMIT 5;
```

This SQL query retrieves the names of the 5 employees with the highest salary by joining the "employees" and "salary" tables on the employee ID, ordering the results by salary in descending order, and limiting the output to 5 rows.


In [ ]:
prompt = 'give me the employees graduates in 2017'

In [ ]:
context.append({'role':'user', 'content':f"{prompt}."})

In [ ]:
response = continue_conversation(context)

In [ ]:
print(response)

This is your SQL:
```sql
SELECT e.name
FROM employees e
JOIN studies st ON e.ID_usr = st.ID_usr
WHERE st.Years = '2017';
```

This SQL query retrieves the names of employees who graduated in 2017 by joining the "employees" and "studies" tables on the employee ID and filtering the results to include only those with graduation year 2017.
